In [ ]:
!pip uninstall tensorflow

# // Proceed (y/n)? 나오면 y입력

!pip install tensorflow-gpu==1.14

# //RESTART RUNTIME 클릭 


In [ ]:
import tensorflow as tf
import numpy as np
import json
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import glob

print(tf.__version__)
device_name = tf.test.gpu_device_name()
print(device_name) # /device:GPU:0 나오는 것 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

#### 이미지 확인

In [ ]:
# 본인 이미지 경로에 맞춰 확인! 
img_1 = Image.open("C:/Users/Playdata/Desktop/FinalProject/test/dataset/QC_IMAGE/Training/Image_2/QC_cow_segmentation_2_000375.jpg")

plt.imshow(img_1)
print(img_1.size) 
# 1은 이미지 size가 (1080, 1920)로 맞춰진 것 같은데...?
# (2268, 4032)도 존재...

In [ ]:
print(img_1.size)

In [ ]:
plt.imshow(img_1.resize([128,128]))

In [ ]:
data = img_1.convert("RGB")

In [ ]:
data = data.resize([128,128])
plt.imshow(data)

In [ ]:
data = np.asarray(data)

In [ ]:
print(data.shape)
print(data)

In [ ]:
data.reshape([1,128,128,3])
print(data.shape)

#### 경로 설정

In [ ]:
train_data_dir = "C:/Users/Playdata/Desktop/FinalProject/test/dataset/QC_IMAGE/Training"
test_data_dir = "C:/Users/Playdata/Desktop/FinalProject/test/dataset/QC_IMAGE/Validation"
categories = ['1++','1+','1','2','3']
nb_class=len(categories) # 5

In [ ]:
# 이미지 사이즈 지정
image_w = 128
image_h = 128

pixels = image_w * image_h * 3

#### 데이터 불러오기 

In [ ]:
# train 데이터 불러오기

x_train = []
y_train = []
count_train = [0, 0, 0, 0, 0]

# 폴더안에 있는 데이터 다 가져오기
# enumerate() : 리스트의 순서와 값 반환
for idx, c in enumerate(categories):
#     print(idx, c)
    label = [0 for i in range(nb_class)]
#     print(label)
    label[idx] = 1
#     print(label)

    image_dir = train_data_dir+"/Image_"+c
#     print(image_dir)
    files = glob.glob(image_dir+"/*.jpg")
    print(len(files))
    count_train.append(len(files))
#     print(files)

    # 파일안에 있는 이미지 오픈해서 어레이값으로 변경

    for i, f in enumerate(files):
#         print(i)
        count_train[idx] += 1
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w,image_h))
        data = np.asarray(img)
        x_train.append(data)
        y_train.append(label)
        
# print(count_train)

In [ ]:
# test 데이터 불러오기

x_test = []
y_test = []
count_test = [0, 0, 0, 0, 0]

for idx, c in enumerate(categories):
    label = [0 for i in range(nb_class)]
    label[idx] = 1
    
    image_dir = test_data_dir+"/Image_"+c
    files = glob.glob(image_dir+"/*.jpg")

    count_test.append(len(files))

    for i, f in enumerate(files):
        print(i)
        count_test[idx] += 1
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w,image_h))
        data = np.asarray(img)
        x_test.append(data)
        y_test.append(label)
        
print(count_test)

In [ ]:
print(len(x_train))
print(x_train[0].shape)
print(len(y_train))
print(y_train[0])
print(x_train[0])

In [ ]:
print(len(x_test))
print(len(y_test))

In [ ]:
# 데이터 ndarray 타입으로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

#### 저장

In [ ]:
np.save("C:/Users/Playdata/Desktop/FinalProject/test/dataset/trainX.npy",x_train)
np.save("C:/Users/Playdata/Desktop/FinalProject/test/dataset/trainY.npy",y_train)

In [ ]:
np.save("C:/Users/Playdata/Desktop/FinalProject/test/dataset/testX.npy",x_test)
np.save("C:/Users/Playdata/Desktop/FinalProject/test/dataset/testY.npy",y_test)

In [ ]:
train_x = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/trainX.npy")
train_y = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/trainY.npy")

In [ ]:
test_x = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/testX.npy")
test_y = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/testY.npy")

In [ ]:
print(type(train_x))
print(type(train_y))
print(train_x.shape)
print(train_y.shape)
print(train_x[0])

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(train_x[0])

In [ ]:
print(type(test_x))
print(type(test_y))
print(test_x.shape)
print(test_y.shape)
print(test_x[0])

In [ ]:
# 표준화, 정규화
train_x =  train_x.astype('float32')/255

test_x =  test_x.astype('float32')/255

#### CNN 설계

In [ ]:
# hyper-parameter 설정

learning_rate = 0.0001
training_epochs = 30
batch_size = 1000

In [ ]:
# CNN network 설계
tf.reset_default_graph()

with tf.device("/gpu:0"):
  X = tf.placeholder(tf.float32, [None,128, 128, 3])
  #X_img = tf.placeholder(tf.float32, [None, 128*128*3]) #??X128X128X3 자동 셋팅
  Y = tf.placeholder(tf.float32, [None, 5]) # 몇 개의 데이터가 들어올지 모르기 때문에 None으로 설정
  dropout = tf.placeholder(tf.float32) # Convolution Layer에 적용될 dropout
  dropout2 = tf.placeholder(tf.float32) # Fully-connected Layer에 적용될 dropout

  ##########################test###############################
  # convolution layer1
  W1 = tf.Variable(tf.random_normal([3,3,3,16])) # filter 설정
  B1 = tf.Variable(tf.random_normal([16]))
  L1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME') + B1
  L1 = tf.nn.relu(L1)
  L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')
  L1 = tf.nn.dropout(L1, dropout)

  # convolution layer2
  W2 = tf.Variable(tf.random_normal([3,3,16,32]))
  B2 = tf.Variable(tf.random_normal([32]))
  L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME') + B2
  L2 = tf.nn.relu(L2)
  L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')
  L2 = tf.nn.dropout(L2, dropout)

  # convolution layer3
  W3 = tf.Variable(tf.random_normal([3,3,32,64]))
  B3 = tf.Variable(tf.random_normal([64]))
  L3 = tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME') + B3
  L3 = tf.nn.relu(L3)
  L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')
  L3 = tf.nn.dropout(L3, dropout)

  # convolution layer4
  W4 = tf.Variable(tf.random_normal([3,3,64,64]))
  B4 = tf.Variable(tf.random_normal([64]))
  L4 = tf.nn.conv2d(L3,W4,strides=[1,1,1,1],padding='SAME') + B4
  L4 = tf.nn.relu(L4)
  L4 = tf.nn.max_pool(L4, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')
  L4 = tf.nn.dropout(L4, dropout)
  L4 = tf.reshape(L4, [-1, 8*8*64])

  # Fully_connected layer
  FCW= tf.get_variable("FCW", shape=[8*8*64, 128], initializer = tf.contrib.layers.xavier_initializer())
  FCB = tf.Variable(tf.random_normal([128]))
  y = tf.matmul(L4, FCW) + FCB
  y = tf.nn.relu(y)
  y = tf.nn.dropout(y, dropout2)

  #출력
  FCWO = tf.get_variable("FCWO", shape=[128,5], initializer = tf.contrib.layers.xavier_initializer())
  model =  tf.matmul(y, FCWO)
  hypothesis = tf.nn.softmax(model)


In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [ ]:
# # 세션 열고 초기화
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())


# for epoch in range(10):
#     avg_cost = 0
#     total_batch = int(len(train_x)/batch_size)
#     print(epoch+1, "번째")
#     for i in range(total_batch):
#         # print(i)
#         batch_xs = train_x[batch_size*i : batch_size*(i+1)]
#         #print(len(batch_xs))
#         #print(batch_xs.shape)
#         batch_ys = train_y[batch_size*i : batch_size*(i+1)]
#         #print(len(batch_ys))
#         #print(batch_ys.shape)
#         _, cost_val= sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, dropout : 0.5, dropout2 : 0.75})
#         avg_cost += cost_val
#         # print(cost_val)
#     print('Epoch:','%04d'%(epoch), 'cost: ','{:.9f}'.format(avg_cost/len(train_x)))

# # 정확도 계산
# correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# print('Accuracy:',sess.run(accuracy,feed_dict={X: test_x, Y:test_y, dropout : 1.0, dropout2 : 1.0}))



with tf.Session() as sess:
    with tf.device("/gpu:0"):
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(30):
            avg_cost = 0
            total_batch = int(len(train_x)/batch_size)
            
            for i in range(total_batch):
                batch_xs = train_x[batch_size*i : batch_size*(i+1)]
                batch_ys = train_y[batch_size*i : batch_size*(i+1)]
                _, cost_val= sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, dropout : 0.5, dropout2 : 0.75})
                avg_cost += cost_val
    
            print('Epoch:','%04d'%(epoch), 'cost: ','{:.9f}'.format(avg_cost/len(train_x)))

        correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        total_test_batch = int(len(test_x)/batch_size)
        for i in range(total_test_batch):
            batch_xs = test_x[batch_size*i : batch_size*(i+1)]
            batch_ys = test_y[batch_size*i : batch_size*(i+1)]
            print('Accuracy:',sess.run(accuracy,feed_dict={X: batch_xs, Y:batch_ys, dropout : 1.0, dropout2 : 1.0}))

        # test_accuracy = accuracy.eval(feed_dict = {X: test_x, Y:test_y, dropout : 1.0, dropout2 : 1.0})
        # print(test_accuracy)
            # test_accuracy = accuracy.eval(feed_dict = {X: mnist.test.images, Y:mnist.test.labels, dropout : 1.0 , dropout2 : 1.0})
            # print("test accuracy : ", test_accuracy)

In [ ]:
test_accuracy = accuracy.eval(feed_dict = {X: test_x, Y:test_y, dropout : 1.0, dropout2 : 1.0})
print(test_accuracy)

In [ ]:
# 테스트 데이터 정확도 계산

print('Accuracy:',sess.run(accuracy,feed_dict={X: test_x, Y:test_y, dropout : 1.0, dropout2 : 1.0}))
